# Домашнее задание № 2. Мешок слов

## Задание 1 (3 балла)

У векторайзеров в sklearn есть встроенная токенизация на регулярных выражениях. Найдите способо заменить её на кастомную токенизацию

Обучите векторайзер с дефолтной токенизацией и с токенизацией razdel.tokenize. Обучите классификатор с каждым из векторизаторов. Сравните метрики и выберете победителя. 

(в вашей тетрадке должен быть код обучения и все метрики; если вы сдаете в .py файлах то сохраните полученные метрики в отдельном файле или в комментариях)

In [1]:
import re
from collections import Counter
import pandas as pd
import numpy as np
from razdel import tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score

from IPython.display import Image, display

In [2]:
data = pd.read_csv('labeled.csv')
pd.set_option('display.max_colwidth', None)
display(data.head(5))
print(data.shape)
print(data.toxic.value_counts(normalize=True))

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол, вон, а у хохлов еще хуже. Если бы хохлов не было, кисель их бы придумал.\n",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорбление, а доказанный факт - не-дебил про себя во множественном числе писать не будет. Или мы в тебя верим - это ты и твои воображаемые друзья?\n",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скрипалей отравила Россия? Анализировать и думать пытаешься? Ватник что ли?)\n",1.0


(14412, 2)
0.0    0.66514
1.0    0.33486
Name: toxic, dtype: float64


Обучим классификатор с помощью алгоритма Decision Tree:
- с векторизацией через TfidfVectorizer (дефолтная токенизация)
- с векторизацией через TfidfVectorizer (токенизация через razdel.tokenize)

In [3]:
#train-test split
train, test = train_test_split(data, test_size=0.15, shuffle=True)

Векторизуем тексты с помощью дефолтной токенизации.

In [4]:
#build feature vectors with default tokenization
vectorizer_default = TfidfVectorizer()
X_train = vectorizer_default.fit_transform(train.comment)
X_test = vectorizer_default.transform(test.comment) 
print(X_train.shape, X_test.shape)

(12250, 62004) (2162, 62004)


Векторизуем тексты с помощью токенизации через razdel.tokenize.

In [5]:
#custom tokenizer function (razdel.tokenize)
def razdel_tokenize(string):
    string_tokenized = [token.text for token in list(tokenize(string))]
    return string_tokenized

In [6]:
#build feature vectors with custom tokenization
vectorizer_razdel = TfidfVectorizer(tokenizer=razdel_tokenize, token_pattern=None)
X_train_razdel = vectorizer_razdel.fit_transform(train.comment)
X_test_razdel = vectorizer_razdel.transform(test.comment) 
print(X_train_razdel.shape, X_test_razdel.shape)

(12250, 62912) (2162, 62912)


Задаем целевые переменные.

In [7]:
#target values
y_train = train.toxic.values
y_test = test.toxic.values
print(y_train.shape, y_test.shape)

(12250,) (2162,)


Обучаем классификатор на дефолтной токенизации, смотрим метрики.

In [8]:
#learning with default tokenization
DTC = DecisionTreeClassifier()
DTC.fit(X_train, y_train)

#metrics
DTC_preds = DTC.predict(X_test)
print(classification_report(y_test, DTC_preds))

              precision    recall  f1-score   support

         0.0       0.81      0.80      0.80      1479
         1.0       0.58      0.59      0.59       683

    accuracy                           0.73      2162
   macro avg       0.69      0.70      0.69      2162
weighted avg       0.74      0.73      0.73      2162



Обучаем классификатор на кастомной токенизации, смотрим метрики.

In [9]:
#learning with custom tokenization
DTC.fit(X_train_razdel, y_train)

#metrics
DTC_preds = DTC.predict(X_test_razdel)
print(classification_report(y_test, DTC_preds))

              precision    recall  f1-score   support

         0.0       0.80      0.81      0.81      1479
         1.0       0.59      0.57      0.58       683

    accuracy                           0.74      2162
   macro avg       0.70      0.69      0.69      2162
weighted avg       0.74      0.74      0.74      2162



Метрики почти одинаковые, но заметно, что для класса токсичных комментариев токенизация через razdel.tokenize сработала чуть хуже: несмотря на то, что точность выше на 1 процент, полнота ниже на 2 процента (соответственно, и F1-мера проигрывает стандартной токенизации). Точность и полнота для нетоксичных комментариев почти одинаковая: у razdel.tokenize выше полнота, у стандартной токенизации -- точность. Если задача состоит в том, чтобы лучше определять токсичные комментарии, то имеет смысл воспользоваться стандартной токенизацией. Не исключено также, что при другом классификаторе соотношение метрик для стандартного и заданного вручную токенизатора будет отличаться.

## Задание 2 (3 балла)

Преобразуйте таблицу с абсолютными частотностями в семинарской тетрадке в таблицу с tfidf значениями. (Таблица - https://i.ibb.co/r5Nc2HC/abs-bow.jpg) Формула tfidf есть в семинаре на картнике с пояснениями на английском. 
Считать нужно в питоне. Формат итоговой таблицы может быть любым, главное, чтобы был код и можно было воспроизвести вычисления. 

In [10]:
Image(url="https://i.ibb.co/r5Nc2HC/abs-bow.jpg")

Напишем функцию для подсчета TF-IDF. На вход функции подается список документов, на выходе -- матрица "термин-документ" и список токенов (для визуализации в виде датафрейма).

In [11]:
def tfidf_counter(docs):
    docs_tokenized= [re.findall('\w+', doc) for doc in docs]   #simple tokenizer for documents
    terms = list(set([term for doc in docs_tokenized for term in doc]))  #list of terms
    matrix = np.zeros((len(docs), len(terms))) #set document-term matrix
    for term in terms:
        count = 0
        for doc in docs_tokenized:
            if term in doc:
                count += 1
        matrix[:,terms.index(term)] += count   #filling matrix with document frequency
    matrix = np.log(len(docs)/matrix)    #counting inverse document frequency
    for num, doc in enumerate(docs_tokenized):
            counter = Counter(doc)
            for term in terms:
                matrix[num, terms.index(term)] *= counter[term]/len(doc)   #counting tf-idf
    return matrix, terms        

In [12]:
texts = ['я и ты', 'ты и я', 'я, я и только я', 'только не я', 'он']

df = pd.DataFrame(tfidf_counter(texts)[0], index=texts, columns=tfidf_counter(texts)[1])
display(df)

,я,не,он,и,ты,только
я и ты,0.074381,0.000000,0.000000,0.170275,0.30543,0.000000
ты и я,0.074381,0.000000,0.000000,0.170275,0.30543,0.000000
"я, я и только я",0.133886,0.000000,0.000000,0.102165,0.00000,0.183258
только не я,0.074381,0.536479,0.000000,0.000000,0.00000,0.305430
он,0.000000,0.000000,1.609438,0.000000,0.00000,0.000000


## Задание 3 (2 балла)

Обучите 2 любых разных классификатора из семинара. Предскажите токсичность для текстов из тестовой выборки (используйте одну и ту же выборку для обоих классификаторов) и найдите 10 самых токсичных для каждого из классификаторов. Сравните получаемые тексты - какие тексты совпадают, какие отличаются, правда ли тексты токсичные?

Требования к классификаторам:   
а) один должен использовать CountVectorizer, другой TfidfVectorizer  
б) у векторазера должны быть вручную заданы как минимум 5 параметров  
в) у классификатора должно быть задано вручную как минимум 2 параметра  
г)  f1 мера каждого из классификаторов должна быть минимум 0.75  

Сначала разобьем датасет на обучающую и тестовую выборки (для классификаторов нам необходимы одинаковые выборки).

In [13]:
#train-test split
train, test = train_test_split(data, test_size=0.1, shuffle=True)

Теперь векторизуем комментарии. В качестве настраиваемых параметров для векторайзеров в sklearn возьмем lowercase, tokenizer (вместе с token_pattern), min_df, max_df и max_feautures.

Свой токенизатор нам нужен по следующим причинам:
1. Стандартный токенизатор в sklearn-векторайзерах делит предложения на токены по регулярному выражению r”(?u)\b\w\w+\b”. Если использовать такой токенизатор, то слова, которые пишутся через дефис (например, "Ростов-на-Дону") разобьются на три токена. Для нашей задачи может быть важно сохранять такие слова.
2. К специфике токсичных комментариев можно отнести слова, написанные полностью в верхнем регистре (как показатели экспрессивности высказываний), поэтому такие слова нам важно будет оставить "как есть" -- они могут быть значимыми признаками для моделей. 
3. Вместо стандартной токенизации по словам попробуем использовать стемминг -- он уменьшит количество признаков за счет удаления различных окончаний, но при этом сохранит лексические значения корня.

In [14]:
#custom tokenize function
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer('russian')

def tokenize_text(string):
    string = re.findall(r'\b\w+-?\w+\b', string)
    string_case= []
    for token in string:
        if token.isupper():
            string_case.append(token)
        else:
            string_case.append(token.lower())
    string_case = [stemmer.stem(token) for token in string_case]
    return string_case

При подборе параметров min_df и max_df выяснилось следующее:
1. Параметр max_df не имеет смысла брать большой: в датасете нет токенов, которые встречались бы более, чем в 30% документов (при max_df=0.3 из словаря токенов удаляется только одно слово). Оставим параметр на значении 0.05 -- тогда из датасета удаляется 321  слово (то есть мы не берем в расчет слова, которые встречаются в каждом двадцатом документе).
2. По параметру min_df можно сделать вывод о том, что в комментариях довольно разнообразная лексика. С параметром min_df=2 уже игнорируется чуть более половины токенов, что значит, что в каждом втором комментарии встречается уникальное слово (и это при том, что токенами считаются не полноценные слова, а только основы). Чтобы нам хватило признаков для обучения, оставим этот параметр на значении 2.
3. При значительном уменьшении параметра max_features метрики становятся хуже (скорее всего, модели не хватает данных для обучения), поэтому этот параметр оставим приближенным к размеру обучающей выборки (12000).

Так как мы используем свой токенизатор, параметры token_pattern и lowercase должны быть со значением False.

In [15]:
#build feature vectors with CountVectorizer
count_vectorizer = CountVectorizer(lowercase=False, tokenizer=tokenize_text, token_pattern=False, max_df=0.05, min_df=2, max_features=12000)
X_train_count = count_vectorizer.fit_transform(train.comment)
X_test_count = count_vectorizer.transform(test.comment) 
print(X_train_count.shape, X_test_count.shape)

(12970, 12000) (1442, 12000)


In [16]:
#build feature vectors with TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(lowercase=False, tokenizer=tokenize_text, token_pattern=False, max_df=0.05, min_df=2, max_features=12000)
X_train_tfidf = tfidf_vectorizer.fit_transform(train.comment)
X_test_tfidf = tfidf_vectorizer.transform(test.comment) 
print(X_train_tfidf.shape, X_test_tfidf.shape)

(12970, 12000) (1442, 12000)


Задаем целевые переменные.

In [17]:
#target values
y_train = train.toxic.values
y_test = test.toxic.values
print(y_train.shape, y_test.shape)

(12970,) (1442,)


Векторы частотности будем использовать для обучения наивного байесовского классификатора (он хорошо подходит для классификации дискретных признаков). В качестве вручную заданных параметров будем использовать alpha и class_prior.

Для того, чтобы подобрать оптимальный параметр alpha, используем np.arrange из возможных значений alpha и посмотрим, при каком из них классификатор дает высокое значение F1-меры для тестовой выборки (заодно посмотрим на соответствующие значения accuracy).

In [18]:
#set a list of alpha (from 0.01 to 100 with step=0.1)
list_alpha = np.arange(0.01, 100, 0.1)
#set lists for scores
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
f1_test = np.zeros(len(list_alpha))

In [19]:
#for every alpha train model and save score values
count = 0
for alpha in list_alpha:
    NB = MultinomialNB(alpha=alpha)
    NB.fit(X_train_count, y_train)
    
    score_train[count] = NB.score(X_train_count, y_train)    
    score_test[count]= NB.score(X_test_count, y_test)
    f1_test[count] = f1_score(y_test, NB.predict(X_test_count))
    
    count += 1

In [20]:
#save values to dataframe
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, f1_test])
models = pd.DataFrame(data=matrix, columns=['alpha', 'train_accuracy', 'test_accuracy', 'test_F1-measure'])
#display best test F1-meausure and its alpha
display(models.sort_values('test_F1-measure', ascending=False).head(5))

,alpha,train_accuracy,test_accuracy,test_F1-measure
7,0.71,0.928142,0.877254,0.814660
2,0.21,0.931534,0.877254,0.814271
8,0.81,0.927988,0.876560,0.813808
6,0.61,0.928296,0.876560,0.813417
11,1.11,0.925906,0.877254,0.812301


Лучшее значение F1-меры у классификатора при alpha=0.71.

Для параметра class_prior установим значение, равное распределению классов во всем датасете -- 2:1.

In [21]:
#Naive Bayes Classifier (on count vectors)
NB_count = MultinomialNB(alpha=0.71, class_prior=[0.66, 0.33])
NB_count.fit(X_train_count, y_train)

#metrics
NB_count_preds = NB_count.predict(X_test_count)
print(classification_report(y_test, NB_count_preds))

              precision    recall  f1-score   support

         0.0       0.91      0.91      0.91       962
         1.0       0.82      0.81      0.81       480

    accuracy                           0.88      1442
   macro avg       0.86      0.86      0.86      1442
weighted avg       0.88      0.88      0.88      1442



Теперь на векторах TF-IDF обучим классификатор с помощью логистической регрессии. В качестве вручную заданных параметров будем использовать C (интенсивность регуляризации), penalty (метод регуляризации), solver (алгоритм оптимизации). Для solver и penalty параметры выставим 'liblinear' (подходит для небольших датасетов и бинарной классификации) и 'l2' соответственно. Параметр С попробуем подобрать аналогично подбору alpha для наивного байеса.

In [22]:
#set a list of C (from 0.1 to 50 with step=0.1)
list_C = np.arange(0.1, 50, 0.1)
#set lists for scores
score_train = np.zeros(len(list_C))
score_test = np.zeros(len(list_C))
f1_test = np.zeros(len(list_C))

In [23]:
#for every C train model and save score values
count = 0
for C in list_C:
    LR = LogisticRegression(C=C, max_iter=1000, penalty='l2', solver='liblinear')
    LR.fit(X_train_tfidf, y_train)
    
    score_train[count] = LR.score(X_train_tfidf, y_train)    
    score_test[count]= LR.score(X_test_tfidf, y_test)
    f1_test[count] = f1_score(y_test, LR.predict(X_test_tfidf))
    
    count += 1

In [24]:
#save values to dataframe
matrix = np.matrix(np.c_[list_C, score_train, score_test, f1_test])
models = pd.DataFrame(data=matrix, columns=['C', 'train_accuracy', 'test_accuracy', 'test_F1-measure'])
#display best test F1-meausure and its С
display(models.sort_values('test_F1-measure', ascending=False).head(10))

,C,train_accuracy,test_accuracy,test_F1-measure
64,6.5,0.970702,0.874480,0.800441
69,7.0,0.971473,0.873786,0.800000
68,6.9,0.971396,0.873786,0.800000
70,7.1,0.971627,0.873786,0.800000
71,7.2,0.971935,0.873786,0.800000
72,7.3,0.972167,0.873786,0.800000
65,6.6,0.971010,0.873786,0.799559
66,6.7,0.971010,0.873786,0.799559
76,7.7,0.972938,0.873093,0.799122
63,6.4,0.970316,0.873786,0.799117


In [25]:
#LogisticRegression (on tfidf vectors)
LR_tfidf = LogisticRegression(C=6.5, penalty='l2', solver='liblinear')
LR_tfidf.fit(X_train_tfidf, y_train)

#metrics
LR_tfidf_preds =LR_tfidf.predict(X_test_tfidf)
print(classification_report(y_test, LR_tfidf_preds))

              precision    recall  f1-score   support

         0.0       0.88      0.93      0.91       962
         1.0       0.85      0.76      0.80       480

    accuracy                           0.87      1442
   macro avg       0.87      0.84      0.85      1442
weighted avg       0.87      0.87      0.87      1442



Теперь посмотрим с помощью метода predict_proba на самые токсичные комментарии для каждого классификатора.

In [26]:
#probas for NB
NB_probas = pd.DataFrame(test.comment)
NB_probas['toxic_probability'] = NB_count.predict_proba(X_test_count)[:,1]

display(NB_probas.sort_values(['toxic_probability'], ascending=False).head(10))

,comment,toxic_probability
1823,"Стас, никому, кроме тебя и армии твоих подсосов(которые представляют собой типичный дегенеративный биомусор, ведущийся на любые скандалы-интриги), твои ролики нахуй не нужны. Серьёзно, ты сделал новости с целью показать, что такое говно может делать любой, а аудитория осталась на том же уровне, ведь людям извне ты не интересен. Да ещё и просит не подписываться, чтобы такую-то годноту ложкой хлебать подольше. Ты обосрался, стал посмешищем для абсолютно всех ютуберов, которые не являются полными ебланами. Тот же Хованский не ссыт тебе на ебало только потому, что ты вертишься с ним в одной компании, иногда даже лично пересекаетесь. Приятно было слышать, как он говорил, что отстреливался бы от таких, как ты, из огнестрела, стараясь забрать с собой побольше коммунистов, когда они придут его оаскулачиапть? Он открыто хуесосил людей и за меньшие грехи. Сложи 2 и 2, как он к тебе относится на самом деле. После чего ты сделал ещё более смешной ролик, где истеришь как побитая шлюха во время ПМС. Я ПОДЕБИЛ, А ЕСЛИ ВЫ НЕ ПОНЯЛИ ЭТОГО, ТО ВЫ ТУПЫЕ . Ты мог хотя бы сам его посмотреть перед заливом? Мне даже рофлить над тобой расхотелось, из смешного дегенерата, ты стал жалким дурачком. Это как смеятся над роликами, где контуженные ветераны пытаются ходить под клубную музыку. Над неполноценными смеятся плохо, даже стыдно стало. Я не утрирую. Просто посмотри на себя, Стас. Ну правда. Банишь людей в группе за лвйки и одно упоминание стрима. Ты делаешь всё, в чём самый отбитый и дегенеративный либераст обвиняет совок и сверкаешь разорванным очком. Никто тебя несправедливо не обсирал. Что на стриме по поводу дат, ну ты же сам проебался. На подкасте сообщил, что не будешь стримить. Если ты не был уверен, то зачем это говорить? А если был, почему не сообщил Маргиналу сразу же? Твоё слово в целом не стоит нихуя. Обещаешь не банить-куча удалённых комментов. Обещаешь стрим-не идёшь. Обещаешь что-то ещё, всегда проёбываешься, всё чаще на нарушение обещания тебе нужно в районе секунды-дня. Стоит ли удивлятся, что тебе за это прилетело? Когда то должно было. Ты сам срёшь себе в штаны, не злись, когда на это указывают пальцем.",1.0
5913,"А сейчас смотрит хуйню всякую с пидорасом звоновым, В оправдание пидораса Звонова можно хотя бы сказать, что последний только и делал контент у чма на стримах: начиная с походов на спасс, новых гостей, научпопвидосов, заканчивая пародией на Что Где Когда. Вообще, после окончания лета я как-то переосмыслил свое отношение ко многим членам инвалидной конфы, да и к самому Маргиналу. Он часто пользуется контентом и талантами своих гостей, приглашая их за нихуя на стримы. При этом хуесосит их же за спиной и на других стримах, из-за чего всё, что гости в итоге получают в ответ за совместные стримы - это говно от маргинальных подписчиков у себя в фиде. И при этом Маргиналу хватает наглости потом ныть, что никто из нормальных людей к нему на стрим не хочет идти, увидав его стенку. Ему не приходит в голову, что поощряя токсичность в отношении своих оппонентов и гостей на стримах он, хотя и обретает ореол интеллектуального стримера, сливателя всяго и вся живого на руси, но по итогу все больше окукливается в собственной же аргументации и аудитории. Заметьте, что те, с кем он еще год назад дебатировал и спорил, сейчас просто избегают споров на стримах, просто не хотя засёра своей ленты (Левин тому ярчайщий пример. Или, уж простите, Звонов). Если раньше Маргоша был способен в самоиронию благодаря тем же убермемес, где замечали и косяки самого Маргинала, то с киком Льва Шойгу начался просто процесс окукливания. Теперь хуесосить Маргинала - запрещено, запрещено даже обсуждать и замечать тупость маргинала. Потому что, как говорит чмо: влияет на просмотры , ололо. По сути, запретив пятнать свою репутацию, но продолжая пятнать репутацию других (и друзей, и врагов) чмо сейчас пытается, сознательно или нет, построить образ эдакого интеллектуала-сливателя, каким бы мечтал стать Ларин. Только вот этот образ Мар

In [27]:
#probas for LR
LR_probas = pd.DataFrame(test.comment)
LR_probas['toxic_probability'] = LR_tfidf.predict_proba(X_test_tfidf)[:,1]

display(LR_probas.sort_values(['toxic_probability'], ascending=False).head(10))

,comment,toxic_probability
2061,"Какие же хохлы незалежные дегенераты, пиздец просто.\n",0.999866
6314,"Плюсы: -Какие же хохлы дегенераты, пиздец просто. Минусы: -Какие же хохлв дегенераты, пиздец просто.\n",0.999832
6497,"null 0 Сука, какие же коммибляди тупые.\n",0.999716
13642,мандан против хохлов?\n,0.999679
2019,Когда тред прощания с хохлами будет?\n,0.999664
6432,Потому что сенегалец лучше хохла. Хохлы вообще не люди\n,0.999638
13889,"Какие блять передергивания? Ты дебил блять зашел на шок-доску и удивляешься что над тобой издеваются. Тут нет твоих друзей, рачье тупорылое, тут тебя все ненавидят. Как же печет от таких необучаемых ебланов. Ты ковбой, твою жену ебут где-то нахуй, а дети гибнут на Украине. Понял, быдло ты ебаное?\n",0.999492
2011,"Нахуй иди, я тебе весь тред что ли читать буду? Пидор, бешбармак тебе в хычин!\n",0.999453
5720,"Свин, иди нахуй. Можно хоть один тред без политоты? Даже тебя хуеосить не хочется, давай лучше про няшек-фигуристок говорить.\n",0.999221
2080,"НУ ВСЕ, сука говго щас модераторам пишу твою тему удалят нахуй к хуям говняным\n",0.999064


Для логистической регрессии все 10 топ-токсичных комментариев действительно токсичные, для наивного байеса есть один, который кажется странным, но при этом не является оскорбительным. Заметное отличие -- длина комментариев (наивный байес к самым токсичным комментариям относит довольно длинные документы, в отличие от логистической регрессии).

## *Задание 4 (2 балла)

Для классификаторов LogisticRegression, Decision Trees, Naive Bayes, Random Forest найдите способ извлечь важность признаков для предсказания токсичного класса. Сопоставьте полученные числа со словами (или нграммами) в словаре и найдите топ - 5 "токсичных" слов для каждого из классификаторов. 

Важное требование: в топе не должно быть стоп-слов. Для этого вам нужно будет правильным образом настроить векторизацию.

**Векторизация**

Для начала разбиваем датасет на обучающую и тестовую выборки (данные будут одинаковые для всех классификаторов).

In [28]:
#train-test split
train, test = train_test_split(data, test_size=0.1, shuffle=True)

Для того, чтобы избавиться от стоп-слов, в векторайзер можно подать соответствующий список. Готовый список стоп-слов есть, например, в nltk, но он неполный, поэтому нам необходимо его расширить за счет слов, которые в нашем датасете встречаются одинаково часто в токсичных и нетоксичных комментариях. Для этого сформируем для каждого класса комментариев список наиболее часто встречающихся слов (возьмем по 300 для каждого класса, ранжировать будем по относительной частоте) и добавим к списку из nltk те из них, которые встречаются в обоих списках. 

In [29]:
#nltk list of stop-words
stops = stopwords.words('russian')

In [30]:
#function for counting frequency of words in corpus
def words_counter(corpus):
    all_tokens = []
    for document in corpus:
        for token in re.findall(r'\b\w+-?\w+\b', document.lower()):
            all_tokens.append(token)
    words = Counter(all_tokens)
    for word in words:
        words[word] /= len(words)  
    return words

In [31]:
#adding frequent words to stop-words list
toxic = [word[0] for word in words_counter(data.comment[data.toxic==1]).most_common(300)]
not_toxic = [word[0] for word in words_counter(data.comment[data.toxic==0]).most_common(300)]

count = 0
for word in list(set(toxic) & set(not_toxic)):
    if word not in stops:
        stops.append(word)
        count += 1
print(count)

88


Список стоп-слов увеличился на 88 токенов. Этот список подадим как параметр stop_words для векторайзера. Чтобы уменьшить количество признаков, установим параметр min_df (при изменении max_df количество признаков почти не меняется -- это связано с тем, что мы уже используем очистку от стоп-слов).

In [32]:
#build feature vectors
vectorizer = CountVectorizer(stop_words=stops, min_df=3)
X_train = vectorizer.fit_transform(train.comment)
X_test = vectorizer.transform(test.comment) 
print(X_train.shape, X_test.shape)

(12970, 13827) (1442, 13827)


In [33]:
#target values
y_train = train.toxic.values
y_test = test.toxic.values
print(y_train.shape, y_test.shape)

(12970,) (1442,)


**Важность признаков для Logistic Regression**

У логистической регрессии важность признаков показывают коэффициенты признаков в функции. Они вызываются свойством coef_.

In [34]:
#learning
LR = LogisticRegression()
LR.fit(X_train, y_train)

LogisticRegression()

In [35]:
#importances for features
importances = pd.DataFrame(vectorizer.get_feature_names_out(), columns=['feature'])
importances['LR'] = LR.coef_[0]

display(importances.sort_values(['LR'], ascending=False).head(5))

,feature,LR
13143,хохлов,3.117413
13147,хохлы,2.851370
2347,дебил,2.796493
13150,хохол,2.184779
11769,сука,2.176660


**Важность признаков для Decision Trees**

Для просмотра важности признаков у деревьев решений есть свойство feature_importances_.

In [36]:
#learning
DT = DecisionTreeClassifier()
DT.fit(X_train, y_train)

DecisionTreeClassifier()

In [37]:
#importances for features
importances['DT'] = DT.feature_importances_
display(importances.sort_values(['DT'], ascending=False).head(5))

,feature,LR,DT
13147,хохлы,2.851370,0.015308
6092,нахуй,1.514811,0.012597
13143,хохлов,3.117413,0.012341
817,блядь,1.638883,0.008233
818,блять,1.940730,0.007834


**Важность признаков для Naive Bayes**

У наивного байеса важность признаков можно извлечь из feature_log_prob_ (свойство возвращает numpy.ndarray, нам необходим второй, для токсичного класса).

In [38]:
NB = MultinomialNB()
NB.fit(X_train, y_train)

MultinomialNB()

In [39]:
importances['NB'] = NB.feature_log_prob_[1]
display(importances.sort_values(['NB'], ascending=False).head(5))

,feature,LR,DT,NB
6092,нахуй,1.514811,0.012597,-6.053295
13147,хохлы,2.851370,0.015308,-6.133338
817,блядь,1.638883,0.008233,-6.220349
818,блять,1.940730,0.007834,-6.251121
13143,хохлов,3.117413,0.012341,-6.338132


**Важность признаков для Random Forest**

Для просмотра важности признаков у деревьев решений есть свойство feature_importances_.

In [40]:
RF = RandomForestClassifier()
RF.fit(X_train, y_train)

RandomForestClassifier()

In [41]:
importances['RF'] = RF.feature_importances_
display(importances.sort_values(['RF'], ascending=False).head(5))

,feature,LR,DT,NB,RF
13147,хохлы,2.851370,0.015308,-6.133338,0.010295
13143,хохлов,3.117413,0.012341,-6.338132,0.009757
6092,нахуй,1.514811,0.012597,-6.053295,0.007266
817,блядь,1.638883,0.008233,-6.220349,0.005644
11769,сука,2.176660,0.006317,-6.688335,0.004958


В целом, списки топ-5 токсичных слов для каждого классификатора примерно похожи, все из них действительно обладают высокой степенью токсичности. У деревьев решений,  наивного байеса и случайного леса списки очень похожи (у деревьев решений и наивного байеса списки идентичные).